In [ ]:
import os

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%time
folder = 'assets/'
title_filename = 'title.basics.tsv.gz'
if os.path.exists(folder + title_filename):
    films = pd.read_csv(folder + title_filename, sep='\t', na_values=['\\N'])
else:
    print(f'{title_filename} does not exist, using url to get data')
    films = pd.read_csv(f'https://datasets.imdbws.com/{title_filename}', sep='\t', na_values=['\\N'])
    
print(f'{films.shape=}')

rating_filename = 'title.ratings.tsv.gz'
if os.path.exists(folder + rating_filename):
    ratings = pd.read_csv(folder + rating_filename, sep='\t', na_values=['\\N'])
else:
    print(f'{rating_filename} does not exist, using url to get data')
    ratings = pd.read_csv(f'https://datasets.imdbws.com/{rating_filename}', sep='\t', na_values=['\\N'])
    
print(f'{ratings.shape=}')

# будем рассматривать только фильмы с оценками
films = pd.merge(films, ratings, on='tconst', how='inner')

crew_filename = 'title.crew.tsv.gz'
if os.path.exists(folder + crew_filename):
    crew = pd.read_csv(folder + crew_filename, sep='\t', na_values=['\\N'])
else:
    print(f'{crew_filename} does not exist, using url to get data')
    crew = pd.read_csv(f'https://datasets.imdbws.com/{crew_filename}', sep='\t', na_values=['\\N'])
    
print(f'{crew.shape=}')

names_filename = 'name.basics.tsv.gz'
if os.path.exists(folder + names_filename):
    names = pd.read_csv(folder + names_filename, sep='\t', na_values=['\\N'])
else:
    print(f'{names_filename} does not exist, using url to get data')
    names = pd.read_csv(f'https://datasets.imdbws.com/{names_filename}', sep='\t', na_values=['\\N'])

print(f'{names.shape=}')


In [ ]:
films.head()

In [ ]:
films.startYear.hist()

### ограничивания датасет выборкой из 10^5 фильмов, чтобы выполнять команды было не долго
Когда станет понятно, что нужно посчитать, надо будет на полном датасете считать

In [ ]:
# df = films.sample(100000)
df = films.copy()
df.shape

In [ ]:
df = pd.merge(df, crew, how='left', on='tconst')
df.head()

In [ ]:
# у фильма может быть несколько режиссеров
# строка с запятыми будет преобразована в список строк
df.directors = df.directors.apply(lambda x: x.split(',') if not pd.isna(x) else np.nan)
df.writers = df.writers.apply(lambda x: x.split(',') if not pd.isna(x) else np.nan)

df['number_of_directors'] = df.directors.apply(lambda x: len(x) if not np.all(pd.isna(x)) else np.nan)

# у фильма может быть несколько жанров (до трех)
# строка с запятыми будет преобразована в список строк
# main_genre это первый жанр
df['genres_lst'] = df.genres.apply(lambda x: 
                                   x.split(',') if not pd.isna(x) else np.nan)

df['main_genre'] = df.genres_lst.apply(lambda x: 
                                       x[0] if not np.all(pd.isna(x)) else np.nan)

df.head()

In [ ]:
sns.boxplot(df, x='main_genre', y='averageRating', hue='isAdult')
plt.xticks(rotation=90);

In [ ]:
sns.boxplot(df, x='main_genre', y='numVotes', hue='isAdult')
plt.xticks(rotation=90);
plt.yscale('log')

In [ ]:
# пусть в каждой строке будет указан только один режиссер
# тогда фильму с двумя режиссерами, будет соответствовать две строки
exploded = df.explode('directors')
print(f'{exploded.shape=}')
exploded.head()

In [ ]:
# добавляется информация о человеке
films_with_directors = pd.merge(exploded, names, how='left', 
                                left_on=['directors'], right_on=['nconst'])
print(f'{films_with_directors.shape=}')
films_with_directors.head()

In [ ]:
# сколько режиссеров сняли по какому количеству фильмов
# индекс это количество фильмов
# directors это сколько режиссеров сняли такое количество фильмов
films_with_directors.groupby('directors').averageRating.count().reset_index().groupby('averageRating').count()

In [ ]:
films_with_directors.info()

нулевых значений в поле `genres` не очень много, можно было бы их и удалить

In [ ]:
# может быть жанр фильма закодировать?
pd.get_dummies(films_with_directors.main_genre)

In [ ]:
films_with_directors.head()

In [ ]:
number_of_films = pd.pivot_table(films_with_directors, columns='main_genre', index=['nconst', 'primaryName'],
                          values=['averageRating'], aggfunc='count').fillna(0)
number_of_films.head()

In [ ]:
number_of_films = pd.pivot_table(films_with_directors, columns='main_genre', 
                                 index=['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession'],
                          values=['averageRating'], aggfunc=['count']).fillna(0).reset_index()
number_of_films.head()

In [ ]:
average_score_of_films = pd.pivot_table(films_with_directors, columns='main_genre', 
                                        index=['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession'],
                          values=['averageRating'], aggfunc=['count', 'mean']).reset_index()#.to_flat_index()
average_score_of_films.head()

In [ ]:
# average_score_of_films.columns = np.logical_or(average_score_of_films.columns.get_level_values(1),
#                                                average_score_of_films.columns.get_level_values(0))

average_score_of_films.columns = average_score_of_films.columns.get_level_values(0) + \
                                               average_score_of_films.columns.get_level_values(2)

average_score_of_films.columns

In [ ]:
average_score_of_films.head()

In [ ]:
%%time
# создается dataframe для режиссера
aggregated = films_with_directors.groupby(['directors'] + list(names.columns)).agg(list).reset_index()
print(f'{aggregated.shape=}')
aggregated.head()

In [ ]:
from collections import Counter

In [ ]:
aggregated['average_score'] = aggregated.averageRating.apply(np.mean)
aggregated['most_common_genre'] = aggregated.main_genre.apply(lambda x: Counter(x).most_common()[0][0])

In [ ]:
if 'writers' in aggregated.columns:
    aggregated = aggregated.drop(columns=['writers', 'genres_lst'])

In [ ]:
aggregated.head()

In [ ]:
aggregated['best_movie'] = aggregated.apply(lambda x: x.primaryTitle[np.argmax(x.averageRating)], axis=1)
aggregated['most_viewed'] = aggregated.apply(lambda x: x.primaryTitle[np.argmax(x.numVotes)], axis=1)

In [ ]:
aggregated['number_of_movies'] = aggregated.averageRating.apply(len)

In [ ]:
aggregated.head()

In [ ]:
merged = pd.merge(aggregated, average_score_of_films, on=['nconst', 'primaryName', 'birthYear',
                                                          'deathYear','primaryProfession'])
merged.head()

In [ ]:
merged.to_csv('assets/directors_rating_by_genre.tsv', sep='\t', index=False)

# Заключение

`merged` датасет по режиссерам.

`average_score` средняя оценка фильмов режиссера

`averageRating` список из оценок фильмов

`most_common_genre` жанр, в котором чаще всего снимает режиссер

`most_viewed` фильм, для которого пользователи IMDb больше всех оценок поставили

`number_of_movies` общее количество фильмов

В колонке с жанром приведена средняя оценка режиссера по фильмам в соответствующем жанре.

# Вопросы:
1. Режиссер чаще снимает один или с кем-то?
2. Какой лучший фильм у режиссера?
3. Какая средняя оценка фильмов для самого частого жанра у этого режиссера?
4. Как составить рейтинг режиссера, чтобы учесть популярность фильма, оценку и жанр? Нужно ли учитывать год?
5. Как у режиссера может быть очень много фильмов?

# Графики

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(aggregated, x='birthYear', y='average_score', color='most_common_genre', 
                 size='number_of_movies', 
                 custom_data=['primaryName', 'number_of_movies', 'most_common_genre', 'best_movie'],
                labels={'birthYear':'Год рождения','average_score':'Рейтинг',
                            'most_common_genre':'Самый популярный жанр'})

fig.update_traces( hovertemplate='<i>Режиссер</i>: %{customdata[0]}'+
                '<br>Средняя оценка: %{y:.2f}<br>'+
                'Количество фильмов:%{customdata[1]}<br>'+
                'Самый частый жанр:%{customdata[2]}<br>' + 
                'Лучший фильм:%{customdata[3]}'
    )

fig.show()
fig.write_html('assets/output/directors.html')

In [ ]:
genre = 'Comedy'

fig = px.scatter(merged, x='birthYear', y='meanComedy', color='average_score', 
                 size='number_of_movies', 
                 custom_data=['primaryName', 'countComedy', 'most_common_genre', 'best_movie'],
                labels={'birthYear':'Год рождения','meanComedy':'Рейтинг', 'average_score':'Рейтинг по всем фильмам',
                            'most_common_genre':'Самый популярный жанр'},
                 title=genre)

fig.update_traces( hovertemplate='<i>Режиссер</i>: %{customdata[0]}'+
                '<br>Средняя оценка по жанру: %{y:.2f}<br>'+
                'Количество фильмов жанра:%{customdata[1]}<br>'+
                'Самый частый жанр:%{customdata[2]}<br>' + 
                  'Лучший фильм:%{customdata[3]}'
    )

fig.show()
fig.write_html(f'assets/output/directors_{genre}.html')
# сохранение в png слишком медленное, дождаться не удалось
# fig.write_image(f'assets/output/directors_{genre}.png')

!pip install -U kaleido

!pip install dash

In [ ]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Рейтинг режиссеров'),
    dcc.Graph(id="scatter-plot"),
    html.P("Фильтр по количеству фильмов режиссера:"),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=1228, step=1,
        marks={x: str(x) for x  in range(50, 1000, 50)},
        value=[0, 1300]
    ),
])


@app.callback(
    Output("scatter-plot", "figure"), 
    Input("range-slider", "value"))
def update_bar_chart(slider_range):
    #df = px.data.iris() # replace with your own data source
    low, high = slider_range
    mask = (aggregated['number_of_movies'] > low) & (aggregated['number_of_movies'] < high)
    fig = px.scatter(aggregated[mask], x='birthYear', y='average_score', color='most_common_genre', 
                 size='number_of_movies', custom_data=['primaryName', 'number_of_movies', 'most_common_genre'],
                     labels={'birthYear':'Год рождения','average_score':'Рейтинг',
                            'most_common_genre':'Самый популярный жанр'},
            )
    
    
    fig.update_traces( hovertemplate='<i>Режиссер</i>: %{customdata[0]}'+
                '<br>Средняя оценка: %{y:.2f}<br>'+
                'Количество фильмов:%{customdata[1]}<br>'+
                'Самый частый жанр:%{customdata[2]}'
    )
    fig.update_xaxes(range=[aggregated.birthYear.min(),aggregated.birthYear.max()])
    fig.update_yaxes(range=[aggregated.average_score.min(),aggregated.average_score.max()])
    return fig


app.run_server(debug=True, host= '127.0.0.1', port=8899)